In [ ]:
import pandas as pd
import os
from gensim.models import word2vec, KeyedVectors
import pandas as pd
# Read data into papers
papers = pd.read_csv('C:/Users/hp/Desktop/finalyear/RNN-LDA-Topic-Labeling/Initiation-practice/CSV/T.csv')
# Print head
papers.head()

In [ ]:
papers = papers.drop(columns=['UserName', 'ScreenName', 'TweetAt'], axis=1).sample(100)
papers.head()

In [ ]:
import re
papers['OriginalTweet_processed'] = \
papers['OriginalTweet'].map(lambda x: re.sub('[,\.!?]', '', x))

In [ ]:
# Convert the titles to lowercase
papers['OriginalTweet_processed'] = \
papers['OriginalTweet_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['OriginalTweet_processed'].head()

In [ ]:
from wordcloud import WordCloud
long_string = ','.join(list(papers['OriginalTweet_processed'].values))
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
wordcloud.generate(long_string)
wordcloud.to_image()

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = papers.OriginalTweet_processed.values.tolist()
data_words = list(sent_to_words(data))


data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

In [ ]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word,num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]